In [4]:
import pyreadstat
import os 
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#lines below let allow multiple results from a line of code to be shown e.g. df.head() + df.columns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
os.chdir('C:\\Users\\Sam Cannon\\Desktop\\R\\Data Sets\\new_datasets_7e\\new_datasets_7e')

In [6]:
#SPSS reading module, need to replace column names though
import savReaderWriter as s
df= pd.DataFrame(list(s.SavReader('C:\\Users\\Sam Cannon\\Desktop\\R\\Data Sets\\new_datasets_7e\\new_datasets_7e\\Lesson 25 Data File 1.sav')))

In [8]:
#rename columns
df = df.set_axis(['Group', 'Difference'], axis=1, inplace=False)

In [14]:
df

,Group,Difference
0,1.0,12.0
1,1.0,-2.0
2,1.0,9.0
3,1.0,3.0
4,1.0,3.0
5,1.0,0.0
6,1.0,3.0
7,1.0,2.0
8,1.0,4.0
9,1.0,1.0


__ANOVA with statsmodels__

- Code was adapted from: https://www.marsja.se/four-ways-to-conduct-one-way-anovas-using-python/ and https://pythonfordatascience.org/anova-python/#test
- Using statsmodels, we get a bit more information and enter the model as a regression formula. The general input using this method looks like this:
    - model_name = ols('outcome_variable ~ group1 + group2 + groupN', data=your_data).fit()

- If you dummy code the groups, you have to not include 1 of the groups in the formula. This group’s data will still get captured in the model’s intercept and is the base (control) group. If you use the following method of entering the formula Python takes care of this for you.

In [21]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#have to use C() with the control group
mod = ols('Difference ~ C(Group)', data=df).fit()
                
aov_table = sm.stats.anova_lm(mod, typ=2)
print(aov_table)

          sum_sq    df         F   PR(>F)
C(Group)   205.4   2.0  4.835891  0.01603
Residual   573.4  27.0       NaN      NaN


In [22]:
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Difference   R-squared:                       0.264
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     4.836
Date:                Mon, 22 Jul 2019   Prob (F-statistic):             0.0160
Time:                        14:20:45   Log-Likelihood:                -86.824
No. Observations:                  30   AIC:                             179.6
Df Residuals:                      27   BIC:                             183.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           3.5000      1.457      2.402      0.023       0.510       6.490
C(Group)[T.2.0]    -5.6000      2.061     -2.717      0.011      -9.829      -1.371
C(Group)[T.3.0]    -5.5000      2.061     -2.669      0.013      -9.729      -1.271
==============================================================================
Omnibus:                        3.156   Durbin-Watson:                   2.135
Prob(Omnibus):                  0.206   Jarque-Bera (JB):                2.729
Skew:                           0.652   Prob(JB):                        0.256
Kurtosis:                       2.304   Cond. No.                         3.73
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""